In [111]:
import math
import random
import numpy as np
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [112]:
stop_words = set(stopwords.words('english'))
print(stop_words) # Needed?

{"that'll", "didn't", 'above', "it's", 'this', 'these', 'be', 'up', 'nor', 'while', 'had', 'during', 'am', "haven't", 'been', 'them', 'now', 'doesn', 'myself', 'we', 't', 'mightn', 'yourself', 'did', 'is', 'there', 'than', 'which', 'an', 'down', "won't", 'hasn', 'because', 'd', 'its', 'him', 'isn', 'aren', 'yours', 'mustn', 'other', 'and', 'how', "mightn't", 'm', 'he', 'any', 'our', 'being', 'were', "needn't", 'with', "hadn't", 'needn', 'who', 'below', 'after', 'a', 'my', 'at', 'ours', 'whom', 'that', 'about', 'on', 'so', 'i', 'each', 'yourselves', 'as', 'what', 'no', 'not', 'll', 'for', "isn't", "shouldn't", 'very', 'from', 'here', 'once', 'same', "wouldn't", "mustn't", 'your', 'out', 'doing', 'they', 'before', 'himself', 'between', 'again', "couldn't", 'the', 'themselves', 'have', 'shan', 'more', "wasn't", 'why', 'shouldn', 'ourselves', 'wouldn', 'will', 'she', 'those', 'where', "doesn't", 'haven', 'was', 're', 'too', 'ma', 'of', 'y', "aren't", 'you', 'few', 'me', "you're", 'has', 't

In [113]:
def peek(filename, size, enc='utf-8'):
    """
    Print out the first X lines in the file.
    """
    if size <= 0:
        print("Size must be greater than zero!")
        return

    with open(filename, encoding=enc) as f:
        for i,line in enumerate(f):
            if (size >= 0) and (i >= size):
                break
            if i == 0: # Skip top line.
                continue
            print(line.strip())

In [118]:
def readDataFromFile(data, filename, size=-1, enc='utf-8'):
    """
    Read and preprocess a certain amount of lines from a file.
    """
    if size == 0:
        print("Nothing to read! Set size=-1 to read the whole file. AT YOUR OWN RISK.")
        return

    with open(filename, encoding=enc) as f:
        for i,line in enumerate(f):
            if (size >= 0) and (i >= size):
                break
            if i == 0: # Skip top line.
                continue
            
            line = line.strip().replace('"', '').replace('\'', '').split(' ')
            a = line[0].replace('<', '').replace('>', '').split('/')[-1].replace('_', ' ')
            b = line[1].replace('<', '').replace('>', '').split('/')[-1].replace('_', ' ')
            c = ""
            if line[2].startswith('<'):
                c = line[2].replace('<', '').replace('>', '').split('/')[-1][len('Category:'):].replace('_', ' ')
            else:
                c = ' '.join(line[2:]).replace('@en .', '')
            
            if not a in data:
                data[a] = {'abstract': [], 'subject': []} # Default dict. structure for an entity? (' '.join(field) will be done before indexing elasticsearch)

            data[a][b].append(c)

In [115]:
def loadData(size):
    """
    Parse dictionary data, size = lines to load from each file.
    """
    kv = {}
    files = [
        'datasets/DBpedia/long_abstracts_en.ttl',
        'datasets/DBpedia/article_categories_en.ttl'
    ]
    for o in files:
        readDataFromFile(kv, o, size) # Update this func. if more files are added to the list in case of parsing bugs!
    return kv

In [116]:
peek('datasets/DBpedia/infobox_properties_en.ttl', 10)

<http://dbpedia.org/resource/Anarchism> <http://dbpedia.org/property/voy> "no"^^<http://www.w3.org/1999/02/22-rdf-syntax-ns#langString> .
<http://dbpedia.org/resource/Anarchism> <http://dbpedia.org/property/n> "no"^^<http://www.w3.org/1999/02/22-rdf-syntax-ns#langString> .
<http://dbpedia.org/resource/Anarchism> <http://dbpedia.org/property/v> "no"^^<http://www.w3.org/1999/02/22-rdf-syntax-ns#langString> .
<http://dbpedia.org/resource/Anarchism> <http://dbpedia.org/property/b> "Subject:Anarchism"^^<http://www.w3.org/1999/02/22-rdf-syntax-ns#langString> .
<http://dbpedia.org/resource/Anarchism> <http://dbpedia.org/property/s> "Portal:Anarchism"^^<http://www.w3.org/1999/02/22-rdf-syntax-ns#langString> .
<http://dbpedia.org/resource/Anarchism> <http://dbpedia.org/property/d> "Q6199"^^<http://www.w3.org/1999/02/22-rdf-syntax-ns#langString> .
<http://dbpedia.org/resource/A> <http://dbpedia.org/property/name> "Latin Capital Letter A"^^<http://www.w3.org/1999/02/22-rdf-syntax-ns#langString> .

In [103]:
peek('datasets/DBpedia/long_abstracts_en.ttl', 10)

<http://dbpedia.org/resource/Animalia_(book)> <http://dbpedia.org/ontology/abstract> "Animalia is an illustrated children's book by Graeme Base. It was originally published in 1986, followed by a tenth anniversary edition in 1996, and a 25th anniversary edition in 2012. Over three million copies have been sold. A special numbered and signed anniversary edition was also published in 1996, with an embossed gold jacket."@en .
<http://dbpedia.org/resource/Actrius> <http://dbpedia.org/ontology/abstract> "Actresses (Catalan: Actrius) is a 1997 Catalan language Spanish drama film produced and directed by Ventura Pons and based on the award-winning stage play E.R. by Josep Maria Benet i Jornet. The film has no male actors, with all roles played by females. The film was produced in 1996."@en .
<http://dbpedia.org/resource/Alain_Connes> <http://dbpedia.org/ontology/abstract> "Alain Connes (French: [alɛ̃ kɔn]; born 1 April 1947) is a French mathematician, currently Professor at the Collège de Fra

In [101]:
peek('datasets/DBpedia/anchor_text_en.ttl', 10)

<http://dbpedia.org/resource/Computer_accessibility> <http://dbpedia.org/ontology/wikiPageWikiLinkText> "Computer accessibility"@en .
<http://dbpedia.org/resource/History_of_Afghanistan> <http://dbpedia.org/ontology/wikiPageWikiLinkText> "History of Afghanistan"@en .
<http://dbpedia.org/resource/Geography_of_Afghanistan> <http://dbpedia.org/ontology/wikiPageWikiLinkText> "Geography of Afghanistan"@en .
<http://dbpedia.org/resource/Demographics_of_Afghanistan> <http://dbpedia.org/ontology/wikiPageWikiLinkText> "Demographics of Afghanistan"@en .
<http://dbpedia.org/resource/Communications_in_Afghanistan> <http://dbpedia.org/ontology/wikiPageWikiLinkText> "Communications in Afghanistan"@en .
<http://dbpedia.org/resource/Transport_in_Afghanistan> <http://dbpedia.org/ontology/wikiPageWikiLinkText> "Transport in Afghanistan"@en .
<http://dbpedia.org/resource/Afghan_Armed_Forces> <http://dbpedia.org/ontology/wikiPageWikiLinkText> "Afghan Armed Forces"@en .
<http://dbpedia.org/resource/Foreign

In [102]:
peek('datasets/DBpedia/article_categories_en.ttl', 10)

<http://dbpedia.org/resource/A> <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:ISO_basic_Latin_letters> .
<http://dbpedia.org/resource/A> <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Vowel_letters> .
<http://dbpedia.org/resource/Achilles> <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Characters_in_the_Iliad> .
<http://dbpedia.org/resource/Achilles> <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Demigods_of_Classical_mythology> .
<http://dbpedia.org/resource/Achilles> <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Kings_of_the_Myrmidons> .
<http://dbpedia.org/resource/Achilles> <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Greek_mythological_hero_cult> .
<http://dbpedia.org/resource/Achilles> <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:People_of_the_Trojan_War> .
<http://dbpedia.org/resource/Achilles>

In [106]:
peek('datasets/DBpedia/instance_types_en.ttl', 10)

<http://dbpedia.org/resource/Anarchism> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Thing> .
<http://dbpedia.org/resource/Achilles> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Thing> .
<http://dbpedia.org/resource/Autism> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> .
<http://dbpedia.org/resource/Alabama> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/AdministrativeRegion> .
<http://dbpedia.org/resource/Abraham_Lincoln> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/OfficeHolder> .
<http://dbpedia.org/resource/Abraham_Lincoln__1> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/TimePeriod> .
<http://dbpedia.org/resource/Abraham_Lincoln__2> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/TimePeriod> .
<http://dbpedia.org/resource/Abraham_Lincoln__3>

In [107]:
peek('datasets/DBpedia/mappingbased_literals_en.ttl', 10)

<http://dbpedia.org/resource/Autism> <http://dbpedia.org/ontology/icd10> "F84.0" .
<http://dbpedia.org/resource/Autism> <http://dbpedia.org/ontology/icd9> "299.00" .
<http://dbpedia.org/resource/Autism> <http://dbpedia.org/ontology/omim> "209850"^^<http://www.w3.org/2001/XMLSchema#integer> .
<http://dbpedia.org/resource/Autism> <http://dbpedia.org/ontology/meshId> "D001321" .
<http://dbpedia.org/resource/Autism> <http://xmlns.com/foaf/0.1/name> "Autism"@en .
<http://dbpedia.org/resource/Alabama> <http://xmlns.com/foaf/0.1/name> "State of Alabama"@en .
<http://dbpedia.org/resource/Alabama> <http://dbpedia.org/ontology/demonym> "Alabaman or Alabamian"@en .
<http://dbpedia.org/resource/Alabama> <http://dbpedia.org/ontology/areaTotal> "1.35765E11"^^<http://www.w3.org/2001/XMLSchema#double> .
<http://dbpedia.org/resource/Alabama> <http://dbpedia.org/ontology/areaLand> "1.31426E11"^^<http://www.w3.org/2001/XMLSchema#double> .


In [108]:
peek('datasets/DBpedia/mappingbased_objects_en.ttl', 10)

<http://dbpedia.org/resource/Anarchism> <http://www.w3.org/2000/01/rdf-schema#seeAlso> <http://dbpedia.org/resource/Anarchist_terminology> .
<http://dbpedia.org/resource/Anarchism> <http://www.w3.org/2000/01/rdf-schema#seeAlso> <http://dbpedia.org/resource/Anarchism> .
<http://dbpedia.org/resource/Anarchism> <http://www.w3.org/2000/01/rdf-schema#seeAlso> <http://dbpedia.org/resource/France> .
<http://dbpedia.org/resource/Anarchism> <http://www.w3.org/2000/01/rdf-schema#seeAlso> <http://dbpedia.org/resource/Violence> .
<http://dbpedia.org/resource/Anarchism> <http://www.w3.org/2000/01/rdf-schema#seeAlso> <http://dbpedia.org/resource/Education> .
<http://dbpedia.org/resource/Alabama> <http://dbpedia.org/ontology/country> <http://dbpedia.org/resource/United_States> .
<http://dbpedia.org/resource/Alabama> <http://dbpedia.org/ontology/language> <http://dbpedia.org/resource/English_American> .
<http://dbpedia.org/resource/Alabama> <http://dbpedia.org/ontology/capital> <http://dbpedia.org/res

In [117]:
kv=loadData(500)
kv['Achilles']

{'abstract': ['In Greek mythology, Achilles (/əˈkɪliːz/; Ancient Greek: Ἀχιλλεύς, Akhilleus, pronounced [akʰilːéu̯s]) was a Greek hero of the Trojan War and the central character and greatest warrior of Homers Iliad. His mother was the nymph Thetis, and his father, Peleus, was the king of the Myrmidons. Achilles’ most notable feat during the Trojan War was the slaying of the Trojan hero Hector outside the gates of Troy. Although the death of Achilles is not presented in the Iliad, other sources concur that he was killed near the end of the Trojan War by Paris, who shot him in the heel with an arrow. Later legends (beginning with a poem by Statius in the 1st century AD) state that Achilles was invulnerable in all of his body except for his heel. Because of his death from a small wound in the heel, the term Achilles heel has come to mean a persons point of weakness.'],
 'subject': ['Characters in the Iliad',
  'Demigods of Classical mythology',
  'Kings of the Myrmidons',
  'Greek mythol

In [110]:
list(kv.keys())[:15]

['Animalia (book)',
 'Actrius',
 'Alain Connes',
 'Agricultural science',
 'International Atomic Time',
 'Astronomer',
 'An American in Paris',
 'Allan Dwan',
 'List of Atlas Shrugged characters',
 'Achilles',
 'A',
 'Arraignment',
 'Answer',
 'Appellate court',
 'Albedo']